In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os

class MyDatabase:
    # http://docs.sqlalchemy.org/en/latest/core/engines.html
    """
    Custom class for instantiating a SQL Alchemy connection. Configured here for SQLite, but intended to be flexible.
    Credit to Medium user Mahmud Ahsan:
    https://medium.com/@mahmudahsan/how-to-use-python-sqlite3-using-sqlalchemy-158f9c54eb32
    """
    DB_ENGINE = {
       'sqlite': 'sqlite:////{DB}'
    }

    # Main DB Connection Ref Obj
    db_engine = None
    def __init__(self, dbtype, 
                 username='', password='', 
                 dbname='',path=os.getcwd()+'/'):
        dbtype = dbtype.lower()
        if dbtype in self.DB_ENGINE.keys():
            engine_url = self.DB_ENGINE[dbtype].format(DB=path+dbname)
            self.db_engine = create_engine(engine_url)
            print(self.db_engine)
            
        else:
            print("DBType is not found in DB_ENGINE")
            
db = MyDatabase('sqlite',dbname='mlb.db',path="/Users/schlinkertc/code/MLB/")


Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)


In [2]:
db.db_engine.table_names()

['actions',
 'credits',
 'game_batting_stats',
 'game_pitching_stats',
 'game_player_links',
 'game_team_links',
 'games',
 'hit_data',
 'matchups',
 'pitch_data',
 'pitches',
 'players',
 'plays',
 'runners',
 'team_records',
 'teams',
 'venues']

In [29]:
d = pd.read_sql('select * from pitch_data limit 10',db.db_engine)
p = pd.read_sql('select * from pitches limit 10',db.db_engine)

Columns to choose from:

In [13]:
p.columns

Index(['gamePk', 'atBatIndex', 'playEndTime', 'index', 'playId', 'pitchNumber',
       'startTime', 'endTime', 'isPitch', 'type', 'details_description',
       'details_code', 'details_ballColor', 'details_isInPlay',
       'details_isStrike', 'details_isBall', 'details_hasReview',
       'count_balls', 'count_strikes', 'details_call_code',
       'details_call_description', 'pfxId', 'details_trailColor',
       'details_type_code', 'details_type_description', 'details_fromCatcher',
       'details_runnerGoing'],
      dtype='object')

In [14]:
d.columns

Index(['gamePk', 'atBatIndex', 'playEndTime', 'index', 'pitchData_startSpeed',
       'pitchData_endSpeed', 'pitchData_strikeZoneTop', 'pitchData_zone',
       'pitchData_typeConfidence', 'pitchData_plateTime',
       'pitchData_extension', 'pitchData_coordinates_aY',
       'pitchData_coordinates_aZ', 'pitchData_coordinates_pfxX',
       'pitchData_coordinates_pfxZ', 'pitchData_coordinates_pX',
       'pitchData_coordinates_pZ', 'pitchData_coordinates_vX0',
       'pitchData_coordinates_vY0', 'pitchData_coordinates_vZ0',
       'pitchData_coordinates_x', 'pitchData_coordinates_y',
       'pitchData_coordinates_x0', 'pitchData_coordinates_y0',
       'pitchData_coordinates_z0', 'pitchData_coordinates_aX',
       'pitchData_breaks_breakY', 'pitchData_breaks_spinRate',
       'pitchData_breaks_spinDirection'],
      dtype='object')

This is slow, how can we make it faster?

In [127]:
stmt = """
SELECT 
    p.atBatIndex,
    p.gamePk,
    p.details_type_description,
    d.pitchData_endSpeed,
    p.pitchNumber,
    plays.result_event,
    p.details_isStrike,
    m.pitcher_id,
    m.batter_id,
    p.count_strikes,
    p.count_balls
FROM 
    pitch_data d
INNER JOIN 
    pitches p
ON
    p.gamePk=d.gamePk
    AND
    p.atBatIndex=d.atBatIndex
    AND
    p.`index`=d.`index`
INNER JOIN 
    plays
ON 
    plays.gamePk = p.gamePk
    AND
    plays.about_endTime=p.playEndTime
INNER JOIN 
    matchups m
    ON 
    m.gamePk=p.gamePk
    AND 
    m.playEndTime=plays.about_endTime
    AND
    m.atBatIndex = plays.about_atBatIndex
WHERE
    p.isPitch == 1
limit 
    1000
"""

In [128]:
df = pd.read_sql(stmt,db.db_engine)

In [130]:
df.head(10)

,atBatIndex,gamePk,details_type_description,pitchData_endSpeed,pitchNumber,result_event,details_isStrike,pitcher_id,batter_id,count_strikes,count_balls
0,0,529870,Four-Seam Fastball,87.7,1,Strikeout,1,622864,434158,1,0
1,0,529870,Four-Seam Fastball,89.0,2,Strikeout,0,622864,434158,1,1
2,0,529870,Changeup,83.9,3,Strikeout,0,622864,434158,1,2
3,0,529870,Two-Seam Fastball,88.3,4,Strikeout,1,622864,434158,2,2
4,0,529870,Four-Seam Fastball,88.9,5,Strikeout,1,622864,434158,2,2
5,0,529870,Slider,81.8,6,Strikeout,1,622864,434158,3,2
6,1,529870,Four-Seam Fastball,88.2,1,Lineout,1,622864,606192,1,0
7,1,529870,Four-Seam Fastball,88.9,2,Lineout,1,622864,606192,2,0
8,1,529870,Four-Seam Fastball,89.4,3,Lineout,1,622864,606192,2,0
9,1,529870,Two-Seam Fastball,90.6,4,Lineout,0,622864,606192,2,1


### Pitch type distribution
Find the pitch types that particular pitcher throws. For the model, this will have to account for data leakage. I can only look at pitches thrown before the pitch in question

In [131]:
stmt = """
SELECT
    p.details_type_description as type,
    count(p.details_type_description) as count
FROM 
    pitches p
INNER JOIN 
    plays
ON 
    plays.gamePk = p.gamePk
    AND
    plays.about_endTime=p.playEndTime
INNER JOIN 
    matchups m
    ON 
    m.gamePk=p.gamePk
    AND 
    m.playEndTime=plays.about_endTime
    AND
    m.atBatIndex = plays.about_atBatIndex
WHERE
    p.isPitch == 1
    AND 
    m.pitcher_id == 622864
GROUP BY 
    p.details_type_description
LIMIT 
    100
"""

In [132]:
pitch_types = pd.read_sql(stmt,db.db_engine)
pitch_types.dropna(inplace=True)
t = pitch_types.transpose()
t.columns=t.loc['type']
t.loc['count']

type
Changeup               54
Four-Seam Fastball     78
Slider                148
Two-Seam Fastball     273
Name: count, dtype: object